In [2]:
%run ../explications.ipynb

# [Le problème du zèbre](https://fr.wikipedia.org/wiki/Int%C3%A9gramme) 
## Génération des explications
### Modélisation

In [3]:
nom = np.array([
         ['anglais', 'espagnol', 'islandais', 'norvegien', 'slovene'],
         ['rouge', 'vert', 'bleu', 'jaune', 'blanc'],
         ['diplomate', 'ingenieur', 'medecin', 'sculpteur', 'violoniste'],
         ['cafe', 'eau', 'lait', 'jusdorange', 'the'],
         ['ane', 'cheval', 'chien', 'renard', 'zebre']
         ], dtype = '<U10')

def model_zebre():
    """
    retourne un quadruplet (vars, facts, constraints, d) tel que
    (vars, facts, constraints) est utilisable dans EXPLANATION
    et d est un dictionnaire : contrainte -> numéro de la contrainte
    """    

    # Variables
    maison = intvar(1, 5, shape=nom.shape, name = 'maison')

    # Pour faciliter l'expression des contraintes :
    for i in range(5): 
        for j in range(5): 
            globals()[nom[i,j]] = maison[i,j]
    
    # faits
    facts = [
        norvegien == 1, #  8.
        lait == 3,      # 11.
    ]

    # contraintes de bijectivité
    cons_b = [AllDifferent(ligne) for ligne in maison]
    
    # constraintes associées aux règles
    d = {
        anglais == rouge :              1, 
        espagnol == chien :             2,
        islandais == ingenieur :        3,
        vert == cafe :                  4,
        vert == blanc - 1 :             5,
        sculpteur == ane :              6,
        diplomate == jaune :            7,
        abs(medecin - renard) == 1 :    9,
        abs(diplomate - cheval) == 1 : 10,
        slovene == the :               12,
        violoniste == jusdorange :     13,
        abs(norvegien - bleu) == 1 :   14,
    }
    cons_r = list(d.keys())
    
    return maison, facts, cons_b + cons_r, d

### Solution et préparation de l'affichage des explications

In [4]:
vars, facts, constraints, _ = model_zebre()
explanation_zebre = EXPLANATION(vars, facts, constraints)
sol = explanation_zebre.solve()

from termcolor import colored

l = 13

def blue(s):
    return colored(s, 'blue', attrs =['bold'], force_color=True)

def green(s):
    return colored(s, 'green', force_color=True)

def pBlue(s):
    print(' ' * (l - len(s)) + colored(s.upper(),'blue', attrs =['bold'],force_color=True), end = '')

def pNormal(s):
    print(' ' * (l - len(s)) + s, end = '')

def print_sol(val, facts = set(), constraints = set()):
    fmt = '{:>13}' * 6
    print(fmt.format(1,2,3,4,5,''))
    sol = np.zeros(shape=nom.shape, dtype = '<U10')
    v =  np.zeros(shape=nom.shape, dtype = 'bool')
    m = np.zeros(shape=nom.shape, dtype = 'bool')
    for fact in facts:
        (i,j),_ = extractFact(fact)
        m[i,j] = True
    for i in range(5):
        for j in range(5):
            k = val[i,j] - 1
            if val[i,j]:
                sol[i, k] = nom[i,j]
                v[i,k] = m[i,j]
        for k in range(5):
            if v[i,k]:
                pBlue(sol[i,k])
            else:
                pNormal(sol[i,k])
        if i in constraints:
            pBlue('◀')
        print('')
        
print_sol(sol)

            1            2            3            4            5             
    norvegien      slovene      anglais    islandais     espagnol
        jaune         bleu        rouge         vert        blanc
    diplomate      medecin    sculpteur    ingenieur   violoniste
          eau          the         lait         cafe   jusdorange
       renard       cheval          ane        zebre        chien


### Explications pour la résolution complète

In [5]:
d = model_zebre()[3]
ESNlist = explanation_zebre.explainFull(verbose = False) # 2 mns

In [6]:
# utile à l'affichage des explications
regles = {                       
    1  : "l'anglais habite la maison rouge",
    2  : "l'espagnol adore son chien",
    3  : "l'islandais est ingénieur",
    4  : "la maison verte sent bon le café",
    5  : "la maison verte est située immédiatement à gauche de la blanche",
    6  : "le sculpteur possède un âne",
    7  : "le diplomate habite la maison jaune",
    8  : "le norvégien habite la première maison à gauche",
    9  : "le médecin habite la maison voisine de celle où demeure le propriétaire du renard",
   10  : "la maison du diplomate est voisine de celle où il y a un cheval",
   11  : "on boit du lait dans la maison du milieu",
   12  : "le slovène boit du thé",
   13  : "le violoniste boit du jus d'orange",
   14  : "le norvégien demeure à côté de la maison bleue"
}

val = np.zeros(shape = nom.shape, dtype = 'int32')

def add_facts(facts):
    for fact in facts:
        (i, j), v = extractFact(fact)
        val[i,j] = v
        
add_facts(facts)
    
for E,S,N in ESNlist:
    constraints = set()
    for r in S:
        if r.name == 'alldifferent':
            constraints.add(extractConstraint(r)[0][0])
    print_sol(val, facts = E, constraints = constraints) 
    print('')
    for r in S:
        if r.name != 'alldifferent':
            print(blue(regles[d[r]]))
    add_facts(N)
    (i, j), v = extractHint(N)
    print('\ndonc '+ green(f'{nom[i,j]} ➔ maison {v}\n') + '_' * 78 + '\n')
    
print_sol(val)

            1            2            3            4            5             
    NORVEGIEN                                                    
                                                                 
                                                                 
                                   lait                          
                                                                 

le norvégien demeure à côté de la maison bleue

donc bleu ➔ maison 2
______________________________________________________________________________

            1            2            3            4            5             
    NORVEGIEN                                                                ◀
                      BLEU                                                   ◀
                                                                 
                                   lait                          
                                                                 

la ma